In [68]:
import sys, time, pickle, torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
sys.path.insert(0, '../../Preprocess')
import numpy as np
import pandas as pd
from preload_models import get_sst2_tok_n_model
from _utils import sample_random_glue_sst2, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info
import torch.nn.functional as F

In [17]:
tens = torch.randn(size=(128, 50, 1024))

In [18]:
tens2 = torch.randn(size=(1024,1))

In [24]:
batch_stack = []
for i in range(tens.shape[0]):
    nu_i = torch.mv(tens[i], tens2.squeeze())
    batch_stack.append(nu_i)

batch_stack = torch.stack(batch_stack, dim=0)

In [25]:
batch_stack.shape

torch.Size([128, 50])

In [35]:
alphas1 = torch.nn.functional.softmax(batch_stack, dim=1)


In [63]:
alphas1.shape

torch.Size([128, 50])

In [61]:
attentioned_samples = []
for sample_alphas, sample_hidden in zip(alphas, tens):
    S_i = torch.mm(sample_alphas.unsqueeze(0), sample_hidden).squeeze(0)
    attentioned_samples.append(S_i) 
attentioned_samples = torch.stack(attentioned_samples, dim=0)
print(attentioned_samples.shape)
    

torch.Size([128, 1024])


In [62]:
attentioned_samples.shape

torch.Size([128, 1024])

In [65]:
protos = torch.randn(size=(34, 1024))

In [77]:
betas = []
for phi_s in attentioned_samples:
    nu_P = torch.sum(F.tanh(phi_s * protos), dim=1)
    beta_p = F.softmax(nu_P)
    betas.append(beta_p)
    
betas = torch.stack(betas, dim=0)
print(f'betas shape: {betas.shape}')

betas shape: torch.Size([128, 34])


/tmp/ipykernel_372870/3368276438.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  beta_p = F.softmax(nu_P)


In [ ]:
tanned.shape

In [ ]:
sst2_data_raw, targets, idxs = sample_random_glue_sst2()

In [ ]:
tokenizer, model = get_sst2_tok_n_model()

In [ ]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [ ]:
from captum.attr import InputXGradient
from captum.attr import visualization 

In [ ]:
input_x_gradient  = InputXGradient(model)

In [ ]:
def generate_record(raw_review, target):
    #tokenizer operations
    tokenized = tokenizer(raw_review, truncation=True, return_offsets_mapping=True)
    offset_mapping = tokenized['offset_mapping']
    conti_map = get_continuation_mapping(offset_mapping)
    input_ids = torch.tensor(tokenized['input_ids']).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    
    #categorizing results
    pred_class = 'Pos' if pred_prob > 0.5 else 'Neg' 
    true_class = 'Pos' if target > 0.5 else 'Neg' 
    
    #attribution algorithm working
    attribution = input_x_gradient.attribute(input_emb)
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Pos' if attr_score > 0.5 else 'Neg'
    convergence_score = None
    
    
    #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
#     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [ ]:
for i, (datum_raw, target) in enumerate(zip(sst2_data_raw, targets), start=1):
    print(f'Raw review: {datum_raw}')
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))

In [ ]:
save_info(idxs, sst2_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='input_x_gradients.pkl')